### Solution to cohort analysis challenge
I believe that ideally this should be solved using Pandas or Spark. Given the restrictions, I opted for solving it using SQL which can handle large datasets and that joining and grouping is naturally fit for SQL.
I would normally add test cases and create a more robust solution, however as I only had a couple hours to complete this, I hope that the general solution approach is acceptable.

In [3]:
import csv, sqlite3
import datetime
from datetime import timedelta

# Load the csv data into the database
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("CREATE TABLE customers (customer_id, signup_date);")
cur.execute("CREATE TABLE orders (id,order_number,customer_id,created);")

with open('customers.csv','r') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['created']) for i in dr]

cur.executemany("INSERT INTO customers (customer_id, signup_date) VALUES (?, ?);", to_db)
con.commit()

with open('orders.csv','r') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['order_number'], i['user_id'], i['created'], ) for i in dr]

cur.executemany("INSERT INTO orders (id,order_number,customer_id,created) VALUES (?, ?, ?, ?);", to_db)
con.commit()

In [4]:
# Find the signup date range:
cur.execute('''SELECT MIN(date(signup_date)), MAX(date(signup_date)) FROM customers''')
result = cur.fetchone()
min_date = datetime.datetime.strptime(result[0], "%Y-%m-%d")
max_date = datetime.datetime.strptime(result[1], "%Y-%m-%d")

In [6]:
# Make a lookup dictionary of users' firt orders
cur.execute(f"SELECT customer_id, MIN(created) FROM orders GROUP BY customer_id")
customers_first = cur.fetchall()
customers_first = dict((x, y) for x, y in customers_first)
# IMHO a better approach than creating a dictionary would be to add a RANK() window function to the query in the next command
# and filter for customers with rank 0 (indicating it's their first order)
# however, window functions are only available in later versions of sqlite3 which is not trivially to setup on Python 3.6

In [8]:
import math 

d = min_date
fp = open('solution.csv', 'w', encoding='utf8')
writer = csv.writer(fp, delimiter='\t', quotechar='"', lineterminator='\n')
# Write header
days_start = 0
csv_row =  ['Cohort', 'Customers']
while days_start < max_elapsed_since_signup:
    days_end = days_start + 6
    csv_row.append(f"{days_start}-{days_end} days")
    days_start += 7

writer.writerow(csv_row)

# Computer cohorts
while d < max_date:
    start = d
    end = d + timedelta(days=6) # For an inclusive range #.strftime("%m/%d/%Y") 
    cur.execute(f"SELECT o.customer_id AS customer_id,c.signup_date, o.created, (Cast(julianday(o.created) - julianday(c.signup_date) AS INTEGER)) AS days_since_signup FROM customers c INNER JOIN orders o ON c.customer_id = o.customer_id WHERE strftime('%s', signup_date) BETWEEN strftime('%s', '{start}') AND strftime('%s', '{end}')")
    all_rows = cur.fetchall()
    customers_count = len(set(map(lambda x: x[0], all_rows)))
    max_elapsed_since_signup = max(list(map(lambda x: x[3], all_rows)))
    date_formatted = f"{start.strftime('%m/%d/%y')}-{end.strftime('%m/%d/%y')}"
    customers_formatted = f"{customers_count} customers"
    csv_row = [date_formatted, customers_formatted]
    days_start = 0
    while days_start < max_elapsed_since_signup:
        days_end = days_start + 6
        in_range = list(filter(lambda x: x[3] >=days_start and x[3] <= days_end, all_rows))
        customers_in_cohort = len(set(map(lambda x: x[0], in_range)))
        customers_in_cohort_first = len(set(map(lambda x: x[0], filter(lambda x: customers_first[x[0]] == x[2], in_range))))
        customers_in_cohort_formated = f"{math.floor(100*(customers_in_cohort/customers_count))}% orders ({customers_in_cohort})"
        customers_in_cohort_first_formated = f"{math.floor(100*(customers_in_cohort_first/customers_count))}% 1st time ({customers_in_cohort})"
        appended = f"{customers_in_cohort_formated} - {customers_in_cohort_first_formated}"
        csv_row.append(appended)
        days_start += 7
    d = end + timedelta(days=1)
    writer.writerow(csv_row)
fp.close()